In [38]:
dates = ["2019-01-19","2019-01-20","2019-01-21","2019-01-22","2019-01-23"]

In [ ]:
def f(row):
    if (row['honeypot'] == 1 or row['blacklist'] == 1) and row['whitelist'] != 1 :
        val = 1
    elif row['whitelist'] == 1 and (row['honeypot'] != 1 and row['blacklist'] != 1):
        val = 0
    else:
        val = -1
    return val

#dir is not keyword
def makemydir(whatever):
  try:
    os.makedirs(whatever)
  except OSError:
    pass
  # let exception propagate if we just can't
  # cd into the specified directory
  os.chdir(whatever)

In [39]:
def labeller(dates):
    import pandas as pd
    import os
    import glob
    import time
    for date in dates:
        file_list = []
        file_list_lab = []
        print(date)
        input_dir = "/scratch/rk9cx/conn_log_labelled_runtime/"+date+"/"
        for file in glob.glob(os.path.join(input_dir, 'anon.conn_tcp.*.log')):
            file_list.append(file.split(date+"/")[1])
        for file in glob.glob(os.path.join(input_dir, 'anon.conn_tcp.*.csv')):
            file_list_lab.append(file.split(date+"/")[1])
        makemydir("/scratch/rk9cx/conn_log_labelled_runtime/labelled_" + date)
        output_dir = "/scratch/rk9cx/conn_log_labelled_runtime/labelled_"+date+"/"
        for file in file_list:
            print("partition")
            start = time.time()
            labelled_logs = pd.read_csv(input_dir+file.strip(".log")+"_LABELS.csv",
                                        usecols=[2,3,4,5])
            labelled_logs.columns = ["src_ip_ext","blacklist","honeypot","whitelist"]
            logs = pd.read_csv(input_dir+file, skiprows=8, 
                               usecols=[0,2,3,4,5,6,7,8,9,10,11,12,14],
                               header = None, delimiter ="\t")
            logs = logs.rename({0:'ts', 2:"src_ip", 3: "src_port", 4:'dest_ip',
                           5:'dest_port', 6:'duration', 7:'src_bytes',
                           8:'dest_bytes', 9:'conn_state',10:'history',
                           11:'src_pkts', 12:'dest_pkts',
                           14:'local'}, axis = 'columns')
            print(logs.shape, labelled_logs.shape)
            merged_logs = pd.concat([logs, labelled_logs],axis=1, sort=False)
            del logs
            del labelled_logs
            merged_logs["label"] = merged_logs.apply(f, axis=1)
            merged_logs = merged_logs[merged_logs.label != -1]
            merged_logs.to_csv(output_dir + file + "finale.csv", index = False)   
            stop = time.time()
            print("Complete. Time elapsed: "+ str(stop - start))
            del merged_logs

In [ ]:
labeller(dates)